In [1]:
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import json
import urllib.parse

import time
import requests
import pandas as pd
import os
import re
import numpy as np

In [2]:
# 1. 사이트 들어가기, Chrome 켜기 
url_car_Grandeur = "http://www.encar.com/dc/dc_carsearchlist.do?carType=kor#!%7B%22action%22%3A%22(And.Hidden.N._.SellType.%EC%9D%BC%EB%B0%98._.(C.CarType.Y._.(C.Manufacturer.%ED%98%84%EB%8C%80._.ModelGroup.%EA%B7%B8%EB%9E%9C%EC%A0%80.)))%22%2C%22toggle%22%3A%7B%7D%2C%22layer%22%3A%22%22%2C%22sort%22%3A%22ModifiedDate%22%2C%22page%22%3A1%2C%22limit%22%3A%2250%22%2C%22searchKey%22%3A%22%22%2C%22loginCheck%22%3Afalse%7D"
driver = wb.Chrome() # Chrome 브라우저 사용
driver.maximize_window() # 창을 최대 크기로 
driver.get(url_car_Grandeur)

car_Grandeur_list = []


extracted_car_names = []
extracted_car_year = []
extracted_car_oilingtype = []
extracted_car_mileages = []
extracted_car_price = []



page = 1



while True:
    print(f"{page}페이지 작업 중...")

    try:
        # 페이지 로딩 대기
        time.sleep(1)

        #--------- 매물 뽑아내기(크롤링 작업 삽입 위치) ----------------
        try:
            car_name_elements = driver.find_elements(By.CSS_SELECTOR, 'div.section.list[data-bind="visible: !!normalSearchResults().sr.length"] tr .cls')
            car_year_elements = driver.find_elements(By.CSS_SELECTOR, 'div.section.list[data-bind="visible: !!normalSearchResults().sr.length"] tr .yer')
            car_oilingtype_elements = driver.find_elements(By.CSS_SELECTOR, 'div.section.list[data-bind="visible: !!normalSearchResults().sr.length"] tr .fue')
            car_mileage_elements = driver.find_elements(By.CSS_SELECTOR, 'div.section.list[data-bind="visible: !!normalSearchResults().sr.length"] tr .km')
            car_price_elements = driver.find_elements(By.CSS_SELECTOR, 'div.section.list[data-bind="visible: !!normalSearchResults().sr.length"] tr .prc')
    
            min_len = min(len(car_name_elements), len(car_mileage_elements), len(car_year_elements), len(car_oilingtype_elements), len(car_price_elements))


            #------------ 뽑아낸 데이터 값을 요소에 할당-------------
            for i in range(min_len):
                name_element = car_name_elements[i]
                year_element = car_year_elements[i]
                oilingtype_element = car_oilingtype_elements[i]
                mileage_element = car_mileage_elements[i]
                price_element = car_price_elements[i]


                #------------- 요소의 텍스트 추출 및 공백 제거해서 다시 할당-----
                name_text = name_element.text.strip()
                year_text = year_element.text.strip()
                oilingtype_text = oilingtype_element.text.strip()
                mileage_text = mileage_element.text.strip() 
                price_text = price_element.text.strip()

                

                #----------------- 데이터를 각 리스트에 하나씩 추가------------
                extracted_car_names.append(name_text)
                extracted_car_year.append(year_text)
                extracted_car_oilingtype.append(oilingtype_text)
                extracted_car_mileages.append(mileage_text)
                extracted_car_price.append(price_text)

                print(f"  {i+1}: 모델명: {name_text},  주행거리: {mileage_text}, 연식 : {year_text}, 연료 : {oilingtype_text}, 가격 : {price_text} 만원")
    
        except Exception as e:
            print(f"차량 이름을 추출하는 중 오류 발생: {e}")

                


        # 다음 페이지 클릭 준비
        page += 1
        selector = f'a[data-page="{page}"]'
        next_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, selector))
        )

        driver.execute_script("arguments[0].click();", next_button)

    except TimeoutException:
        print(f"{page}페이지는 존재하지 않음. 크롤링 종료.")
        break

driver.quit()

1페이지 작업 중...
  1: 모델명: 현대 더 뉴 그랜저 IG,  주행거리: · 33,006km, 연식 : 21/06식(22년형), 연료 : · 가솔린, 가격 : 2,470 만원
  2: 모델명: 현대 더 뉴 그랜저 IG,  주행거리: · 25,822km, 연식 : 22/05식(23년형), 연료 : · 가솔린, 가격 : 2,990 만원
  3: 모델명: 현대 그랜저 (GN7),  주행거리: · 1,808km, 연식 : 24/12식(25년형), 연료 : · 가솔린, 가격 : 3,420 만원
  4: 모델명: 현대 그랜저 하이브리드 (GN7),  주행거리: · 25,626km, 연식 : 22/12식(23년형), 연료 : · 가솔린+전기, 가격 : 4,030 만원
  5: 모델명: 현대 그랜저 IG,  주행거리: · 80,191km, 연식 : 17/09식(18년형), 연료 : · 가솔린, 가격 : 1,579 만원
  6: 모델명: 현대 더 뉴 그랜저 IG 하이브리드,  주행거리: · 60,129km, 연식 : 22/04식, 연료 : · 가솔린+전기, 가격 : 2,750 만원
  7: 모델명: 현대 더 뉴 그랜저 IG,  주행거리: · 82,521km, 연식 : 20/07식, 연료 : · 가솔린, 가격 : 1,870 만원
  8: 모델명: 현대 더 뉴 그랜저 IG,  주행거리: · 133,276km, 연식 : 22/04식, 연료 : · 가솔린, 가격 : 1,960 만원
  9: 모델명: 현대 그랜저 하이브리드 (GN7),  주행거리: · 57,375km, 연식 : 23/05식, 연료 : · 가솔린+전기, 가격 : 3,650 만원
  10: 모델명: 현대 더 뉴 그랜저 IG,  주행거리: · 59,416km, 연식 : 20/09식, 연료 : · LPG(일반인 구입), 가격 : 2,550 만원
  11: 모델명: 현대 더 뉴 그랜저 IG 하이브리드,  주행거리: · 120,669km, 연식 : 20/05식, 연료 : · 가솔린+전기, 가격 : 1,859 만원
  12

In [3]:
# 이제 추출한 데이터를 데이터프레임으로 넣자
if extracted_car_names: # 최소한 차량 이름이라도 추출되었다면
    # 모든 리스트의 길이가 동일하다고 가정하고 DataFrame 생성
    # 만약 길이가 다를 경우, 가장 짧은 길이에 맞춰 슬라이싱해야 합니다.
    min_final_len = min(len(extracted_car_names),
                        len(extracted_car_year),
                        len(extracted_car_oilingtype),
                        len(extracted_car_mileages),
                        len(extracted_car_price))

    # 각 추출한 데이터를 원하는 컬럼명에 리스트를 넣어준다
    data = {
        'model': extracted_car_names,
        'year': extracted_car_year,
        'oilingtype': extracted_car_oilingtype,
        'mileage': extracted_car_mileages,
        'price': extracted_car_price
        
    }

    # pandas DataFrame 생성
    df = pd.DataFrame(data)

    # CSV 파일로 저장
    # 현재 스크립트가 실행되는 디렉토리에 저장됩니다.
    output_filename = "grandeur_raw_data_01.csv"
    # 한글 깨짐 방지를 위해 'utf-8-sig' 인코딩을 사용합니다.
    df.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print(f"\n'{output_filename}' 파일이 성공적으로 생성되었습니다.")
    print("저장된 데이터의 상위 5개 행:")
    print(df.head()) # 상위 5개 데이터 출력하여 확인
else:
    print("\n추출된 차량 이름 데이터가 없습니다. CSS 셀렉터가 올바른지 확인해주세요.")



'grandeur_raw_data_01.csv' 파일이 성공적으로 생성되었습니다.
저장된 데이터의 상위 5개 행:
                model          year oilingtype     mileage  price
0       현대 더 뉴 그랜저 IG  21/06식(22년형)      · 가솔린  · 33,006km  2,470
1       현대 더 뉴 그랜저 IG  22/05식(23년형)      · 가솔린  · 25,822km  2,990
2        현대 그랜저 (GN7)  24/12식(25년형)      · 가솔린   · 1,808km  3,420
3  현대 그랜저 하이브리드 (GN7)  22/12식(23년형)   · 가솔린+전기  · 25,626km  4,030
4           현대 그랜저 IG  17/09식(18년형)      · 가솔린  · 80,191km  1,579


In [4]:
df

,model,year,oilingtype,mileage,price
0,현대 더 뉴 그랜저 IG,21/06식(22년형),· 가솔린,"· 33,006km","2,470"
1,현대 더 뉴 그랜저 IG,22/05식(23년형),· 가솔린,"· 25,822km","2,990"
2,현대 그랜저 (GN7),24/12식(25년형),· 가솔린,"· 1,808km","3,420"
3,현대 그랜저 하이브리드 (GN7),22/12식(23년형),· 가솔린+전기,"· 25,626km","4,030"
4,현대 그랜저 IG,17/09식(18년형),· 가솔린,"· 80,191km","1,579"
...,...,...,...,...,...
10765,현대 그랜저 IG,17/01식,· 가솔린,"· 243,848km",900
10766,현대 그랜저 IG,19/01식,· 가솔린,"· 96,914km","1,990"
10767,현대 그랜저 IG 하이브리드,17/05식(18년형),· 가솔린+전기,"· 169,089km","1,480"
10768,현대 더 뉴 그랜저 IG,21/04식,· 가솔린,"· 26,404km","2,500"


In [10]:
# 공백으로 되어있는 결측치를 NaN값으로 대체하고
# 없애준다
df['price'] = df['price'].replace('', np.nan)
# NaN 값을 'Missing'으로 채우기 (원하는 값으로 변경 가능)
df.dropna(subset="price", inplace = True)
df


,model,year,oilingtype,mileage,price
0,현대 더 뉴 그랜저 IG,21/06식(22년형),· 가솔린,"· 33,006km","2,470"
1,현대 더 뉴 그랜저 IG,22/05식(23년형),· 가솔린,"· 25,822km","2,990"
2,현대 그랜저 (GN7),24/12식(25년형),· 가솔린,"· 1,808km","3,420"
3,현대 그랜저 하이브리드 (GN7),22/12식(23년형),· 가솔린+전기,"· 25,626km","4,030"
4,현대 그랜저 IG,17/09식(18년형),· 가솔린,"· 80,191km","1,579"
...,...,...,...,...,...
10765,현대 그랜저 IG,17/01식,· 가솔린,"· 243,848km",900
10766,현대 그랜저 IG,19/01식,· 가솔린,"· 96,914km","1,990"
10767,현대 그랜저 IG 하이브리드,17/05식(18년형),· 가솔린+전기,"· 169,089km","1,480"
10768,현대 더 뉴 그랜저 IG,21/04식,· 가솔린,"· 26,404km","2,500"


In [11]:
# 연도/월 /연형을 연도 값만 뽑아내서 +1 해서 연형으로 수정

df['year'] = df['year'].astype(str).str[:2].astype(int) + 1
df

,model,year,oilingtype,mileage,price
0,현대 더 뉴 그랜저 IG,22,· 가솔린,"· 33,006km","2,470"
1,현대 더 뉴 그랜저 IG,23,· 가솔린,"· 25,822km","2,990"
2,현대 그랜저 (GN7),25,· 가솔린,"· 1,808km","3,420"
3,현대 그랜저 하이브리드 (GN7),23,· 가솔린+전기,"· 25,626km","4,030"
4,현대 그랜저 IG,18,· 가솔린,"· 80,191km","1,579"
...,...,...,...,...,...
10765,현대 그랜저 IG,18,· 가솔린,"· 243,848km",900
10766,현대 그랜저 IG,20,· 가솔린,"· 96,914km","1,990"
10767,현대 그랜저 IG 하이브리드,18,· 가솔린+전기,"· 169,089km","1,480"
10768,현대 더 뉴 그랜저 IG,22,· 가솔린,"· 26,404km","2,500"


In [12]:
df.to_csv('second_clearly_data_set.csv', index=False, encoding='utf-8-sig')

In [13]:
df

,model,year,oilingtype,mileage,price
0,현대 더 뉴 그랜저 IG,22,· 가솔린,"· 33,006km","2,470"
1,현대 더 뉴 그랜저 IG,23,· 가솔린,"· 25,822km","2,990"
2,현대 그랜저 (GN7),25,· 가솔린,"· 1,808km","3,420"
3,현대 그랜저 하이브리드 (GN7),23,· 가솔린+전기,"· 25,626km","4,030"
4,현대 그랜저 IG,18,· 가솔린,"· 80,191km","1,579"
...,...,...,...,...,...
10765,현대 그랜저 IG,18,· 가솔린,"· 243,848km",900
10766,현대 그랜저 IG,20,· 가솔린,"· 96,914km","1,990"
10767,현대 그랜저 IG 하이브리드,18,· 가솔린+전기,"· 169,089km","1,480"
10768,현대 더 뉴 그랜저 IG,22,· 가솔린,"· 26,404km","2,500"


In [14]:
#모델명 수정
df['model'] = df['model'].str.replace('현대 ', '', regex=False)
df['model'] = df['model'].str.replace(' ', '_', regex=False)
df['model'] = df['model'].str.replace('더', 'the', regex=False)
df['model'] = df['model'].str.replace('그랜저', 'grandeur', regex=False)
df['model'] = df['model'].str.replace('(', '', regex=False)
df['model'] = df['model'].str.replace(')', '', regex=False)
df['model'] = df['model'].str.replace('하이브리드', 'hybrid', regex=False)
df['model'] = df['model'].str.replace('럭셔리', 'luxury', regex=False)
df['model'] = df['model'].str.replace('뉴', 'new', regex=False)

# oilingtype 전부 영어로 변환 
df['oilingtype'] = df['oilingtype'].str.replace('·', '', regex=False)
df['oilingtype'] = df['oilingtype'].str.replace('가솔린', 'gasoline', regex=False)
df['oilingtype'] = df['oilingtype'].str.replace('디젤', 'Diesel', regex=False)
df['oilingtype'] = df['oilingtype'].str.replace('+', '', regex=False)
df['oilingtype'] = df['oilingtype'].str.replace('gasoline전기', 'hybrid', regex=False)
df['oilingtype'] = df['oilingtype'].str.replace('gasolineLPG', 'LPG', regex=False)
df['oilingtype'] = df['oilingtype'].str.replace('LPG(일반인 구입)', 'LPG', regex=False)

# 주행거리 수정
df['mileage'] = df['mileage'].str.replace('km', '', regex=False)
df['mileage'] = df['mileage'].str.replace(',', '', regex=False)
df['mileage'] = df['mileage'].str.replace('·', '', regex=False)

# price 수정
df['price'] = df['price'].str.replace(',', '', regex=False)

# 컬럼 형변환
df['year'] = df['year'].astype(int)
df['mileage'] = df['mileage'].astype(int)
df['price'] = df['price'].astype(int)

# 연형 2자리를 4자리 연도로 변형

# 차량 개수
total_grandeur = 10559

for i in range(total_grandeur):
    original_value = df['year'][i]
    
    if 0 <= original_value < 10:       
        a = str(original_value)
        new_num = int("200" + str(a)) 
        df.at[i, 'year'] = new_num
    elif 10 <= original_value <30:
        a = str(original_value)
        new_num = int("20" + str(a)) 
        df.at[i, 'year'] = new_num
    elif 30 <= original_value < 100 :
        a = str(original_value)
        new_num = int("19" + str(a)) 
        df.at[i, 'year'] = new_num
        

df

,model,year,oilingtype,mileage,price
0,더 뉴 그랜저 IG,22,· 가솔린,"· 33,006km","2,470"
1,더 뉴 그랜저 IG,23,· 가솔린,"· 25,822km","2,990"
2,그랜저 (GN7),25,· 가솔린,"· 1,808km","3,420"
3,그랜저 하이브리드 (GN7),23,· 가솔린+전기,"· 25,626km","4,030"
4,그랜저 IG,18,· 가솔린,"· 80,191km","1,579"
...,...,...,...,...,...
10765,그랜저 IG,18,· 가솔린,"· 243,848km",900
10766,그랜저 IG,20,· 가솔린,"· 96,914km","1,990"
10767,그랜저 IG 하이브리드,18,· 가솔린+전기,"· 169,089km","1,480"
10768,더 뉴 그랜저 IG,22,· 가솔린,"· 26,404km","2,500"


In [18]:
df.to_csv('modified_column.csv', index=False, encoding='utf-8-sig')

In [64]:
df.to_csv('modified_column_second.csv', index=False, encoding='utf-8-sig')

In [68]:
df.to_csv('modified_column_third.csv', index=False, encoding='utf-8-sig')

In [ ]:
df.to_csv('hyundai_fifth.csv', index=False, encoding='utf-8-sig')